In [1]:
from sage.all import *

## Newton-Cotes formulas

In [4]:
def trapezoidal(f, a, b):
    h = b - a
    return h / 2 * (f(x=a) + f(x=b))

trapezoidal(x, 0, 1)

1/2

In [5]:
def simpson(f, a, b):
    h = (b - a) / 2
    return h / 3 * (f(x=a) + 4*f(x=a + h) + f(x=b))

simpson(x, 0, 1)

1/2

In [6]:
def simpson_38(f, a, b):
    h = (b - a) / 3
    return 3 * h / 8 * (f(x=a) + 3*f(x=a + h) + 3*f(x=a + 2*h) + f(x=b))

simpson_38(x, 0, 1)

1/2

In [2]:
def boole(f, a, b):
    h = (b - a) / 4
    return 2 * h / 45 * (7*f(x=a) + 32*f(x=a + h) + 12*f(x=a + 2*h) + 32*f(x=a + 3*h) + 7*f(x=b))

boole(x, 0, 1)

1/2

In [7]:
for rule in [trapezoidal, simpson, simpson_38, boole]:
    print(rule.__name__)
    for k in range(1, 7):
        i = integral(x**k, x, 0, 1)
        q = rule(x**k, 0, 1)
        print(x**k, i, q, i - q)
        if i - q != 0:
            break

trapezoidal
x 1/2 1/2 0
x^2 1/3 1/2 -1/6
simpson
x 1/2 1/2 0
x^2 1/3 1/3 0
x^3 1/4 1/4 0
x^4 1/5 5/24 -1/120
simpson_38
x 1/2 1/2 0
x^2 1/3 1/3 0
x^3 1/4 1/4 0
x^4 1/5 11/54 -1/270
boole
x 1/2 1/2 0
x^2 1/3 1/3 0
x^3 1/4 1/4 0
x^4 1/5 1/5 0
x^5 1/6 1/6 0
x^6 1/7 55/384 -1/2688


## Romberg's method

In [32]:
var('n m')
def romberg(n, m):
    return 1 / 4^(m-1) * (4^m * romberg(n, m-1) - romberg(n-1, m-1))
romberg(n, m)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/chaoses/miniconda3/envs/sage/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_4964/1162797162.py", line 4, in <cell line: 4>
    romberg(n, m)
  File "/tmp/ipykernel_4964/1162797162.py", line 3, in romberg
    return Integer(1) / Integer(4)**(m-Integer(1)) * (Integer(4)**m * romberg(n, m-Integer(1)) - romberg(n-Integer(1), m-Integer(1)))
  File "/tmp/ipykernel_4964/1162797162.py", line 3, in romberg
    return Integer(1) / Integer(4)**(m-Integer(1)) * (Integer(4)**m * romberg(n, m-Integer(1)) - romberg(n-Integer(1), m-Integer(1)))
  File "/tmp/ipykernel_4964/1162797162.py", line 3, in romberg
    return Integer(1) / Integer(4)**(m-Integer(1)) * (Integer(4)**m * romberg(n, m-Integer(1)) - romberg(n-Integer(1), m-Integer(1)))
  [Previous line repeated 2968 more times]
  File "sage/structure/element.pyx", line 1358, in sage.str

TypeError: object of type 'NoneType' has no len()

In [31]:
def romberg(f, a, b, r):
    t = Matrix(RR, r, r)

    n = 1
    h = b - a
    t[0, 0] = h / 2 * (f(x=a) + f(x=b))
    for i in range(1, r):
        t[i, 0] = t[i-1, 0] / 2 + h / 2 * sum(f(x=(k+1/2)*h) for k in range(n))
        h = h / 2
        n *= 2
    
    for j in range(1, r):
        for i in range(j, r):
            t[i, j] = (4**j * t[i, j-1] - t[i-1, j-1]) / (4**j - 1)
    
    print(t)

romberg(sin(x) / x, 1e-100, 1, 4)

[0.920735492403948 0.000000000000000 0.000000000000000 0.000000000000000]
[0.939793284806177 0.946145882273587 0.000000000000000 0.000000000000000]
[0.944513521665390 0.946086933951794 0.946083004063674 0.000000000000000]
[0.945690863582701 0.946083310888472 0.946083069350917 0.946083070387223]


## Composite rule

In [43]:
x = var('x')
f = sin(x) / x
integral(f, x, 0, 1).n()

0.946083070367183

In [23]:
def f(x):
    vals = {
        0: 1.000,
        1/8: 0.997,
        2/8: 0.989,
        3/8: 0.977,
        4/8: 0.959,
        5/8: 0.946,
        6/8: 0.909,
        7/8: 0.877,
        1: 0.841
    }
    return vals[x]

In [30]:
def composite_trapezoidal(f, a, b, n):
    h = (b - a) / n
    return h / 2 * (f(x=a) + 2 * sum(f(x=a + i*h) for i in range(1, n)) + f(x=b))

composite_trapezoidal(f, 0, 1, 8)
#composite_trapezoidal(f, 1e-100, 1, 8).n()

0.946812500000000

In [28]:
def composite_simpson(f, a, b, n):
    h = (b - a) / n
    return h / 6 * (f(x=a) + 4 * sum(f(x=a + (i+1/2)*h) for i in range(n)) + 2 * sum(f(x=a + i*h) for i in range(1, n)) + f(x=b))

composite_simpson(f, 0, 1, 4)
# composite_simpson(f, 1e-100, 1, 4).n()

0.947625000000000